# 操作変数推定量の推定

参考: [計量経済学 (New Liberal Arts Selection)](https://www.amazon.co.jp/%E8%A8%88%E9%87%8F%E7%B5%8C%E6%B8%88%E5%AD%A6-New-Liberal-Arts-Selection/dp/4641053855/ref=sr_1_3?adgrpid=111764532464&dib=eyJ2IjoiMSJ9.wQjA-XsmLxKXc8ABZybC23Xra5aMTm7j0E8ZvykKfwW9j5R6Xs4hFNeNmreAbDkYWa3vuOKBh6E_akbaA4hv-Kvhva08UD3A7uMSUVgwQHtJCb_3vccGkO818r8lfH1wSKV3_IPge6JvcwjdC2L4rDwsGNnqkZd8-b8JkAsjjjYfvBLTyrG179n1D5Q9PYahLtBb7c90opKBxYDcXt_Umd8Mof09asxCWcAr2BBlb4ChsW5nzdRxc3Ov-0mBHLysojmQidaKPg6aTCECrqt7q91-VwmG-Qi7ShHj1pU-gRhmNd_is1_RrVeGe8wBGyrTH8yUBTssL9KeDZsRajsNx65RBeQLisUuF8zSQsSTxhTefEw34jQc2q1LEz91tWPJ-X7i4fwk3qHmLITkM6FjasuusnTctT9Pbx7Jmc3l4HzXVmuP4ULwGNwnXqqrq63S.asv14I1-XrVr4bA_8UVKdgQc06EZUFZ_kDq9l_Mcb3M&dib_tag=se&hvadid=679060345133&hvdev=c&hvlocphy=1009309&hvnetw=g&hvqmt=e&hvrand=9602578875003065665&hvtargid=kwd-930097394579&hydadcr=6190_13376777&jp-ad-ap=0&keywords=%E8%A8%88%E9%87%8F%E7%B5%8C%E6%B8%88%E5%AD%A6&mcid=0ac99b911d5b3cb7a4ee11cd5aee374e&qid=1737268178&sr=8-3)p274より

単回帰モデルにおける操作変数推定量

$$
\hat{\beta}_{1,IV} = \frac{\sum_{i=1}^N (Z_i - \bar{Z})(Y_i - \bar{Y})}{\sum_{i=1}^N (Z_i - \bar{Z})(X_i - \bar{X})}
$$

$$
\hat{\beta}_{0,IV} = \bar{Y} - \hat{\beta}_{1,IV} \bar{X}
$$

## モジュールインポート

In [131]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import japanize_matplotlib
from tqdm import tqdm

from scipy import stats
from statsmodels.api import OLS, add_constant
from statsmodels.stats.weightstats import ttest_ind

## 関数定義

In [132]:
def generate_data(random_seed: int, data_size: int):
    '''効果検証対象となるデータを作成する
       └施策の訴求: 訴求有無を設定し、ユーザーごとにランダムでどちらかを割り当てる
        ├Treatment(T=1): 訴求が行われる
        └Contorl  (T=0): 訴求が行われない
    Args:
        random_seed (int): 乱数のシード値
        data_size   (int): データの行数

    Returns:
        df: 効果検証対象となるデータフレーム
            ├X: 特徴量（サイト回遊意欲・購入意欲の高さを示す交絡因子）
            ├Z : バナー訴求有無（Treatmentは1、Controlは0）
            ├D : バナー閲覧有無
            └Y : 購入有無
    '''

    np.random.seed(random_seed)
    size = data_size
    
    # サイト回遊意欲・購入意欲の高さ
    X = np.random.uniform(0, 5, size=size)
    
    # 訴求有無: ランダムで訴求2パターンが割り当てられるものとする（Treatmentは1、Controlは0）
    Z = np.array([])
    for i in range(size):
        Z_i = np.random.choice(2, size=1, p=[0.5, 0.5])[0]
        Z = np.append(Z, Z_i)
    
    # バナー閲覧確率: サイト回遊意欲・購入意欲とバナーの影響で施策認知確率も高くなるものとする
    d_prob = X / 10 + 0.1*Z + np.random.uniform(-0.04, 0.04, size=size) # バナーによる施策認知への効果: 0.1
    d_prob = np.clip(d_prob, 0, 1)
    D = np.array([])
    for i in range(size):
        D_i = np.random.choice(2, size=1, p=[1-d_prob[i], d_prob[i]])[0]
        D = np.append(D, D_i)

    # 購入の有無: サイト回遊意欲・購入意欲が高いほど、購入確率も高く、またバナー閲覧により購入確率が上がるものとする
    y_prob = X / 10 + np.random.uniform(-0.08, 0.08, size=size) + 0.15*D # バナー閲覧による購入への効果: 0.15
    y_prob = np.clip(y_prob, 0, 1)
    Y = np.array([])
    for i in range(size):
        Y_i = np.random.choice(2, size=1, p=[1-y_prob[i], y_prob[i]])[0]
        Y = np.append(Y, Y_i)
    
    df = pd.DataFrame({'X': X, 'Z': Z, 'D': D, 'Y': Y}).astype('float')
    return df

In [133]:
def calculate_late_estimator(df: pd.DataFrame):
    '''LATE推定量を算出
    
    Args:
        df(pd.DataFrame): 効果検証対象となるデータフレーム

    Returns:
        late_estimator

    '''
    E_Y_Z_1 = df[df['Z']==1]['Y'].mean()
    E_Y_Z_0 = df[df['Z']==0]['Y'].mean()
    E_D_Z_1 = df[df['Z']==1]['D'].mean()
    E_D_Z_0 = df[df['Z']==0]['D'].mean()
    late = (E_Y_Z_1 - E_Y_Z_0) / (E_D_Z_1 - E_D_Z_0)

    return late_estimator

In [134]:
def calculate_iv_estimator(df: pd.DataFrame):
    '''操作変数推定量を算出
    
    Args:
        df(pd.DataFrame): 効果検証対象となるデータフレーム

    Returns:
        iv_estimator

    '''
    average_z = df['Z'].mean()
    average_y = df['Y'].mean()
    average_d = df['D'].mean()
    beta_hat_1_iv = np.sum((df['Z'] - average_z) * (df['Y'] - average_y)) / np.sum((df['Z'] - average_z) * (df['D'] - average_d))
    iv_estimator = beta_hat_1_iv
    
    return iv_estimator

In [135]:
def calculate_iv_estimator_se(df: pd.DataFrame):    
    '''操作変数推定量の標準誤差を算出
    
    Args:
        df(pd.DataFrame): 効果検証対象となるデータフレーム

    Returns:
        iv_estimator_se

    '''
    iv_estimator = calculate_iv_estimator(df=df)
    
    # 操作変数推定量の標準誤差を算出する
    average_y = df['Y'].mean()
    average_d = df['D'].mean()
    beta_hat_0_iv = average_y - iv_estimator * average_d
    mu_hat_i = df['Y'] - beta_hat_0_iv -  iv_estimator * df['D']
    # ルート((1/N)*hoge)のhoge部分の分子と分母を計算する
    # 分子: N-2で割ることで自由度を修正する
    numerator = (((df['Z'] - df['Z'].mean())**2)*(mu_hat_i**2)).sum() / (data_size - 2)
    # 分母
    denominator = ((((df['Z'] - df['Z'].mean()) * (df['D'] - df['D'].mean())) / data_size).sum())**2
    # 操作変数推定量の標準誤差
    se_beta_hat_1_iv = np.sqrt((1/data_size)*(numerator/denominator))
    iv_estimator_se = se_beta_hat_1_iv

    return iv_estimator_se

## LATE推定量と操作変数推定量を比較

In [136]:
# LATE推定量
i = 12345
data_size = 10000
df = generate_data(random_seed=i, data_size=data_size)
late_estimator = calculate_late_estimator(df=df)
print(f'LATE推定量: {late_estimator:,.3f}')

# 操作変数推定量
df = generate_data(random_seed=i, data_size=data_size)
iv_estimator = calculate_iv_estimator(df=df)
print(f'操作変数推定量: {iv_estimator:,.3f}')

LATE推定量: 0.151
操作変数推定量: 0.151


LATE推定量と操作変数推定量は一致する様子

## 操作変数推定量の標準誤差
参考: [計量経済学 (New Liberal Arts Selection)](https://www.amazon.co.jp/%E8%A8%88%E9%87%8F%E7%B5%8C%E6%B8%88%E5%AD%A6-New-Liberal-Arts-Selection/dp/4641053855/ref=sr_1_3?adgrpid=111764532464&dib=eyJ2IjoiMSJ9.wQjA-XsmLxKXc8ABZybC23Xra5aMTm7j0E8ZvykKfwW9j5R6Xs4hFNeNmreAbDkYWa3vuOKBh6E_akbaA4hv-Kvhva08UD3A7uMSUVgwQHtJCb_3vccGkO818r8lfH1wSKV3_IPge6JvcwjdC2L4rDwsGNnqkZd8-b8JkAsjjjYfvBLTyrG179n1D5Q9PYahLtBb7c90opKBxYDcXt_Umd8Mof09asxCWcAr2BBlb4ChsW5nzdRxc3Ov-0mBHLysojmQidaKPg6aTCECrqt7q91-VwmG-Qi7ShHj1pU-gRhmNd_is1_RrVeGe8wBGyrTH8yUBTssL9KeDZsRajsNx65RBeQLisUuF8zSQsSTxhTefEw34jQc2q1LEz91tWPJ-X7i4fwk3qHmLITkM6FjasuusnTctT9Pbx7Jmc3l4HzXVmuP4ULwGNwnXqqrq63S.asv14I1-XrVr4bA_8UVKdgQc06EZUFZ_kDq9l_Mcb3M&dib_tag=se&hvadid=679060345133&hvdev=c&hvlocphy=1009309&hvnetw=g&hvqmt=e&hvrand=9602578875003065665&hvtargid=kwd-930097394579&hydadcr=6190_13376777&jp-ad-ap=0&keywords=%E8%A8%88%E9%87%8F%E7%B5%8C%E6%B8%88%E5%AD%A6&mcid=0ac99b911d5b3cb7a4ee11cd5aee374e&qid=1737268178&sr=8-3)p279より

### 操作変数推定量の標準誤差

$\hat{u}_i$を操作変数推定の残差

$$
\hat{u}_i = Y_i - \hat{\beta}_{0,IV} - \hat{\beta}_{1,IV} X_i
$$
として、

操作変数推定量の標準誤差は
$$
SE(\hat{\beta}_{1,IV}) = \sqrt{\frac{1}{N} \cdot \frac{\sum_{i=1}^N (Z_i - \bar{Z})^2 \cdot \hat{u}_i^2 / N}{\left(\sum_{i=1}^N (Z_i - \bar{Z})(X_i - \bar{X}) / N \right)^2}}
$$
と定義される

※分子のNをN-2に変える自由度修正方法もある

### 操作変数推定量の標準誤差を用いた検定統計量

$$
t = \frac{\hat{\beta}_{1,IV} - \beta_1^{0}}{SE(\hat{\beta}_{1,IV})}
$$

In [137]:
# シミュレーション用のデータを作成
data_size = 10000    # データのサイズ
n_trials = 100       # 試行回数
p_value_thres = 0.05 # 有意水準

significant_list = []
for i in tqdm(range(n_trials)):
    significant_flag = 0
    np.random.seed(i) # ランダムシード
    df = generate_data(random_seed=i, data_size=data_size)
    iv_estimator = calculate_iv_estimator(df=df)
    iv_estimator_se = calculate_iv_estimator_se(df=df)
    
    t_stat = (iv_estimator - 0) / iv_estimator_se
    p_value = 1 - stats.t.cdf(t_stat, df=data_size-2)                 # 片側検定
    # p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=data_size-2)) # 両側検定

    if p_value < p_value_thres:
        significant_flag = 1
    significant_list.append(significant_flag)
significant_ratio = np.array(significant_list).mean()
print(f'有意と判定している割合: {significant_ratio:,.3f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.18it/s]

有意と判定している割合: 0.530


今回のデータでは、サンプルサイズ10,000では検出力が0.530であることがわかった